# Algoritmo de agrupamento - K-means 
***

O algoritmo k-means é um método de clustering utilizado para agrupar dados em k grupos com base na similaridade de suas características. Ele funciona encontrando k centroides (pontos representativos) para cada grupo e atribuindo cada ponto de dados ao centroide mais próximo. Esse processo é repetido até que a posição dos centroides não mude mais. 

O algoritmo k-means é amplamente utilizado em aplicações como análise de dados, segmentação de imagens, etc.



Exemplo prático:
<img src="doc_img/cluster.png" width="300"/>



### Aplicação Financeiras

- Segmentação de clientes financeiros: Pode ser usado para agrupar clientes financeiros semelhantes em grupos com base em suas características financeiras, como renda, despesas, histórico de crédito, etc. Isso permite aos bancos e instituições financeiras personalizar as ofertas de produtos financeiros para cada grupo de clientes.


- Análise de risco: Pode ser usado para avaliar o risco associado a investimentos financeiros, como ações, títulos, etc. Ele pode agrupar investimentos semelhantes em grupos com base em suas características financeiras e histórico de desempenho, o que permite aos investidores tomar decisões informadas sobre o risco e o potencial de retorno.


- Alocação de ativos: Pode ser usado para ajudar os investidores a alocar seus ativos de maneira mais eficiente. Ele pode agrupar ativos semelhantes em grupos com base em suas características financeiras e histórico de desempenho, o que permite aos investidores tomar decisões informadas sobre como alocar seus ativos.

Estes são apenas alguns exemplos das aplicações do algoritmo k-means na área financeira. É importante destacar que o algoritmo deve ser utilizado como parte de uma estratégia de investimento mais ampla e não deve ser visto como a única fonte de informações para tomar decisões financeiras.
*** 

## Implementando algoritmo de K-means
***

Nesta atividade, vamos implementar o algoritmo K-means em um conjunto de dados financeiros de ativos.

Para isso, vamos executar os seguintes passos:


- Coletar dados do dataset com a lista de ativos alvos;
- Coletar dados online do yahoo finances;
- Realizar o enriquecimento de dados;
- Aplicar modelo ML de K-means;
- Visualizar os resultados obtidos;

### Importando as bibliotecas necessárias
***

In [3]:
import pandas as pd
import yfinance as yf
import numpy as np
import warnings

import plotly.express as px  #Criação de graficos dinâmnicos
import plotly.offline as py
import plotly.graph_objects as go #Para criação e concatenização de graficos

from plotly.subplots import make_subplots
from pandas_datareader import data as pdr
from sklearn.cluster import KMeans

import pandas_datareader, sklearn, plotly
warnings.filterwarnings("ignore")
yf.pdr_override()

yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


### Verificando versões das bibliotecas utilizadas
***

In [4]:
print(f''' Relação das bibliotecas e suas respectivas versões
------------------------------------------------------------
  pandas: {pd.__version__}
  numpy: {np.__version__}
  yahoo ficances: {yf.__version__}
  pandas_datareader: {pandas_datareader.__version__}
  plotly: {plotly.__version__}
  sklearn: {sklearn.__version__}
------------------------------------------------------------
        '''
      )

 Relação das bibliotecas e suas respectivas versões
------------------------------------------------------------
  pandas: 2.2.3
  numpy: 2.1.2
  yahoo ficances: 0.2.40
  pandas_datareader: 0.10.0
  plotly: 5.24.1
  sklearn: 1.5.2
------------------------------------------------------------
        


### Coletando dados dos ativos 
***
A primeira etapa do nosso projeto é coletar a lista de ativos alvo.


In [65]:
dados_ativos = pd.read_csv('./dataset/ativos_estrangeiros.csv', sep = ';')

### Vizualizando dataset coletado
***

In [66]:
dados_ativos.head()

,Ativo,Descricao
0,META,Meta Platforms Inc.
1,GOOG,Alphabet Inc (Google).
2,ORCL,Oracle Corporation
3,MSFT,Microsoft Corporation
4,AMZN,Amazon.com Inc.


### Coletando dados adicionais para cada ativo
***
O próximo passo é coletar dados adicionais dos ativos que queremos fazer a clusterização.

In [70]:
# Cria lista de dataframe para coletar e armazenar dados de cada ativo
hoje = pd.Timestamp.today().strftime('%Y-%m-%d')
um_ano_atras = (pd.Timestamp.today() - pd.DateOffset(years=1)).strftime('%Y-%m-%d')
lista_df = []
ativos = dados_ativos['Ativo']
for ativo in ativos:
    try:
        df = pdr.get_data_yahoo(ativo, start=um_ano_atras, end=hoje)
        # Cria nova coluna com o nome do ativo
        df['Ativo'] = ativo
        # Adiciona os dados em uma lista
        lista_df.append(df)
    except e:
        print(f'Foi encontrado um erro no ativo: {ativo} - erro: {e}')
              
df_ativos = pd.concat(lista_df)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


### Verficando quantidade de dados coletados
***

In [71]:
print(f'O dataset coletado possui {len(df_ativos)} linhas')

O dataset coletado possui 2520 linhas


### Vizualisando previa dos dados coletados
***

In [72]:
df_ativos.head()

,Open,High,Low,Close,Adj Close,Volume,Ativo
Date,,,,,,,
2023-10-16,318.640015,321.820007,315.519989,321.149994,320.185944,16536100,META
2023-10-17,318.179993,324.399994,317.299988,324.000000,323.027374,16387800,META
2023-10-18,321.390015,325.940002,315.559998,316.970001,316.018494,16851000,META
2023-10-19,319.880005,321.890015,311.750000,312.809998,311.871002,18709200,META
2023-10-20,314.140015,315.299988,306.470001,308.649994,307.723450,22287400,META


### Selecionando *feature* fechamento ajustado
***
Vamos utilizar a coluna de Adj Close (fechamento ajustado) para enriquecer a nossa base de dados e criar duas novas features para esse estudo. 

-  **Fechamento ajustado:** é o preço de fechamento após ajustes para todas as divisões e distribuições de dividendos aplicáveis.

Antes de criar as novas *features*, primeiramente vamos calcular a diferença de percentual da coluna "Adj CLose" para cada linha com o valor encontrado no nosso dataset.

In [73]:
df_ativos['Percentual Diff'] = df_ativos['Adj Close'].pct_change()

In [74]:
# Visualizando diferença percentual entre os valores
df_ativos.head()

,Open,High,Low,Close,Adj Close,Volume,Ativo,Percentual Diff
Date,,,,,,,,
2023-10-16,318.640015,321.820007,315.519989,321.149994,320.185944,16536100,META,NaN
2023-10-17,318.179993,324.399994,317.299988,324.000000,323.027374,16387800,META,0.008874
2023-10-18,321.390015,325.940002,315.559998,316.970001,316.018494,16851000,META,-0.021697
2023-10-19,319.880005,321.890015,311.750000,312.809998,311.871002,18709200,META,-0.013124
2023-10-20,314.140015,315.299988,306.470001,308.649994,307.723450,22287400,META,-0.013299


### Enriquecendo a base de dados
***

Com a diferença em percentual calculado por dia, vamos criar duas novas *features*:

- Retorno: Indica qual o retorno financeiro o ativo proporciona para a pessoa.
- volatividade: Identifica o quanto que o ativo oscila. É calculado com o desvio padrão.

In [76]:
dias_uteis = 252
retorno = (df_ativos.groupby(['Ativo'])
                             .agg(retorno=('Percentual Diff', 'mean')) * dias_uteis)

volatividade = (df_ativos.groupby(['Ativo'])
                             .agg(volatividade = ('Percentual Diff', 'std')) * np.sqrt(dias_uteis))

analise_ativos = pd.merge(retorno, volatividade, how='inner', on ='Ativo')

In [77]:
# Resetando o index da nova tabela e visualizando dados gerados
analise_ativos.reset_index(inplace=True)
analise_ativos.head()

,Ativo,retorno,volatividade
0,AAPL,0.109327,0.296481
1,AMZN,-0.296948,0.739180
2,GOOG,-0.547868,0.810398
3,IBM,5.485980,4.911182
4,INTC,-1.053571,0.883473


### Analisando o melhor número de cluster através da métrica -  WCSS
***

O WCSS (Within-Cluster Sum of Squared Errors) é uma métrica frequentemente usada na análise de cluster para avaliar o desempenho do algoritmo k-means. A ideia é medir a soma dos erros de distância entre os pontos de dados e seus respectivos centróides de cluster. O objetivo é minimizar essa soma para obter clusters bem formados.

Em resumo, a WCSS representa a soma das distâncias dos pontos de dados aos seus centróides de cluster. Quanto menor o valor de  WCSS, melhor o agrupamento de dados. O gráfico da WCSS vs. número de clusters é usado para determinar o número ótimo de clusters. À medida que o número de clusters aumenta, a WCSS diminui, mas após um certo ponto, a redução na WCSS começa a diminuir e torna-se mais difícil de melhorar o agrupamento. O ponto onde a WCSS começa a diminuir de forma mais moderada é geralmente considerado o número ótimo de clusters.

In [78]:
# Função para calcular valores de WCSS
def calcular_wcss(dados_ativos):
    wcss = []
    for k in range(1,11):
        kmeans = KMeans(n_clusters = k, random_state=0, init='k-means++' )
        kmeans.fit(X=dados_ativos)        
        wcss.append(kmeans.inertia_)
    return wcss

In [79]:
# Seleciona as variáveis que serão utilizadas na clusterização
dados_ativos = analise_ativos[['retorno', 'volatividade']]

# Realiza o cálculo do WCSS
wcss_ativos = calcular_wcss(dados_ativos)

In [80]:
# Visualizando os dados obtidos do WCSS
for i in range(len(wcss_ativos)):
  print(f'O cluster {i+1} possui valor de WCSS de: {wcss_ativos[i]}')

O cluster 1 possui valor de WCSS de: 46.13543493184015
O cluster 2 possui valor de WCSS de: 3.9785174232522817
O cluster 3 possui valor de WCSS de: 1.439825154644919
O cluster 4 possui valor de WCSS de: 0.9527886921640183
O cluster 5 possui valor de WCSS de: 0.6842361240509194
O cluster 6 possui valor de WCSS de: 0.32564390592100045
O cluster 7 possui valor de WCSS de: 0.1321111770631197
O cluster 8 possui valor de WCSS de: 0.05556763601870582
O cluster 9 possui valor de WCSS de: 0.021551181992764623
O cluster 10 possui valor de WCSS de: 0.0


### Visualizando o gráfico de Cálculo de  WCSS VS. Número de Cluster
***

O gráfico de WCSS também conhecido como gráfico do cotovelo ilustra qual o possível número ideal de clusters para o dataset analisado.

In [81]:
grafico_wcss = px.line( x = range(1,11),
                        y = wcss_ativos
                       )
fig = go.Figure(grafico_wcss)

fig.update_layout(title='Calculando o WCSS',
                  xaxis_title= 'Número de clusters',
                  yaxis_title= 'Valor do Wcss', 
                  template =  'plotly_white'
                  ) 

fig.show()

### Aplicando o algoritmo K-mens
***
Após analisar o gráfico de WCSS um possível número de clusters para a base de dados é 5. Diante disso, vamos realizar o agrupamento dos ativos utilizando esse parâmetro. 

In [86]:
kmeans_ativos = KMeans(n_clusters=3, 
                       random_state=0, 
                       init='k-means++')
analise_ativos['cluster'] = kmeans_ativos.fit_predict(dados_ativos)

In [87]:
analise_ativos.head()

,Ativo,retorno,volatividade,cluster
0,AAPL,0.109327,0.296481,0
1,AMZN,-0.296948,0.739180,2
2,GOOG,-0.547868,0.810398,2
3,IBM,5.485980,4.911182,1
4,INTC,-1.053571,0.883473,2


### Visualizando clusters de ativos
***

In [88]:
fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Scatter( x = analise_ativos["volatividade"], 
                          y = analise_ativos["retorno"],
                          name = "", mode = "markers",
                          text = analise_ativos['Ativo'],
                          marker = dict(size = 14, color = analise_ativos["cluster"])
                        )
              )

fig.update_layout( height = 600, width = 900,
                   title_text = "Análise de Clusters",               
                   xaxis_title = "Volatividade",
                   yaxis_title = "Retorno"               
                 )


fig.show()

O gráfico gerado mostra a relação dos agrupamentos dos ativos em relção a sua volatividade Vs. retorno obtido no período analisado. 